In [1]:
import fh
from fh import FH
from treeBuilder import TreeBuilder
import treeBuilder
import PydalChanel as pydal
import copy
import random

# algorithmically generate the tree traversals
# have harmonic tree, and separately have a rhythmic/phrasing tree

# dimensions that can be varried per instrument:
# base rhythm, root note, 
# rhythmic permutation root/traversal 
# melodic variation root/traversal

# 4 values for each of the 6 parameters above 
# have a tree that creates variations of a 4d vector with values [1,2,3,4]
# this tree is the "arrangement tree" 
# - have a traverser on the tree for each parameter
# - the value of the vector for each parameter determines what paramter value each instrument has

# it is important that there is "coordinated change" i.e. that some parameters change
# between instruments in the same way at the same time.
# A way to do this is to have all 6 parameter traversers start at the same node
# in the arrangment tree, and have groups of traversers follow identical paths

# a the vector of the 6 arrangement traverser node positions be a "thematic point"
# pick a handful of thematic points, create a sequence of them
# - that is the "outline" of the composition
# the arangement traversers will pathfind their way between these thematic points,
# with groups traversers moving in identical paths as described above

# how to create these paths?


# ------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------
# the melody playing on each instrument can be represented by a 6 dimensional vector
# (1 value for each of the 6 parameters listed above). 
# A "transformation" is a change to that vector
# Two transformations can be said to be of the same "type" if they affect the same
# set of dimensions in the vector. 
# Two or more instruments simultaneosly undergoing transformations of the same
# "type" is an instance of coordinated change 

# Create a list of matricies with vectors of the above type. they are the "outline" of the composition
# Randomly apply variation functions as described above, slowly attempting to interpolate from
# one matrix to another. 
# Dynamically construct variation trees as a "memory" structure during the interpolation. 
# Revisit older branches/traversals from the varation tree later on, reapplying the same sequence of 
# transformation "types" as used before

lp = FH()
tempo = pydal.tempo

buf2chan = lambda buf, chan: map(lambda elem: elem[:3] + [chan, elem[-1]], buf)
dropOctave = lambda buf, numOctaves: map(lambda elem: [elem[0], max(elem[1]-(12*numOctaves),0)] + elem[2:], buf)

In [ ]:
tree = TreeBuilder([0]*8, fh.transformTranspositionVector, 6, lambda depth: 5)
trees = [copy.deepcopy(tree) for i in range(4)]
trees[0].execute("\/ \/")
trees[1].execute("\/:1 \/")
trees[2].execute("\/:2 \/")
trees[3].execute("\/:3 \/")

traversals = [0]*4
traversals[0] = "+ _ \/*3 -@"
traversals[1] = "+ _ > - + >:2 - + >:3 > -"
traversals[2] = "+ _ \/ ^ \/:1 - + \/:1 \/ > -@"
traversals[3] = "+ \/ \/ ^ > \/ -@"
getVec = lambda treeInd, traversalInd : trees[treeInd].executeStepwise(traversals[traversalInd])



In [ ]:
chan = 0
lp.startChannel(1, lambda buf: fh.shiftTranspose(buf, lp.roots[0], lp.scales[0], getVec(chan, chan)), loop2)
loopTwo = fh.beatShuffle(dropOctave(buf2chan(loop, 2), 2))
lp.startChannel(2, lambda buf: fh.shiftTranspose(buf, lp.roots[0], lp.scales[0], getVec(chan, chan)), loopTwo)

In [ ]:
trees[0].traversalSteps

In [ ]:
lp.stopChannel(1)
lp.stopChannel(2)

In [61]:
inc = treeBuilder.Counter()
tree = TreeBuilder(0, inc, 6, lambda depth: 5)

s = '\/:' + str(1)
tree.execute(s)

treeStep(tree)
tree.currentNode.value



selected choice >:1
selected action >:1


3907

In [77]:
# "traversal" - a "random walk" from a root that generates a tree.
# the random walk should have weights s.t. the root is hit more often
# than gen 1, which is hit more than gen 2 (and so on) and that earlier
# generated siblings are traversed to more than later generated siblings.
# undefined whether newer siblings are more or less common than nodes in 
# the next generation.

# simple implementation - markov model, 5 sibling limit
# .3 - to parent, 
# .25 to child (.06 base, with remainder divided by i*k for ith child (lower i is newer), 
#    k s.t it all adds up, with p(newChild) same as having one newer child
# .45 to sibs (total), with similar division between sibs as for child (can transition to self)
# for root - .2 probability transition to self, per child prob divided as above 
def weighted_choice(choices):
    total = sum(w for w, c in choices)
    r = random.uniform(0, total)
    upto = 0
    for w, c in choices:
        if upto + w >= r:
            #print "selected choice", c
            return c
        upto += w
    assert False, "Shouldn't get here"

def treeStep(arrangementTree, newNodeAllowed=False):
    node = tree.currentNode
    n = len(node.children)
    #returns distribution over move-to-child/new-child
    def childChoices(childTotalP, childBaseP, numChildren, newNodeIncluded): 
        num = numChildren+1 if newNodeIncluded else numChildren
        piece = (childTotalP - childBaseP*(num)) / sum(range(num))
        return [childBaseP + (num-i-1)*piece for i in range(num)] 
    if tree.currentNode == tree.root:
        prepAction = ""
        rootP = 0.2
        pChildren = childChoices(.8, .1, len(node.children))
        choices = [(rootP, '_')] + [(pChildren[i], '\/:'+str(i)) for i in range(n)] 
        if newNodeAllowed:
            choice += [(pChildren[-1], '\/!')]
    else:
        prepAction = "^ \/"
        tree.execute(prepAction) #makes "move to nth sibling" (rather than nth "next") logic cleaner 
        rootP = 0.3
        pChildren = childChoices(.25, .06, len(node.children))
        pSiblings = childChoices(.45, .11, len(node.parent.children))
        choices = [(rootP, '^')] + [(pChildren[i], '\/:'+str(i)) for i in range(n)]  \
            + [(pChildren[i], '>:'+str(i)) for i in range(n)] 
        if newNodeAllowed:
            choice += [(pSiblings[-1], '< >!'), (pChildren[-1], '\/!')]
    action = weighted_choice(choices)
    #print "selected action", action
    tree.execute(action)
    return prepAction + " " + action

In [ ]:
# select major themes as random (6*4) matricies - randomly drop instruments?
# TODO: have some recursively flavored way to create a repetitive sequence of the themes

# iteration - go for 4-8 "matrix-steps" then jump, to next theme, 
# saving both the traversal type list as well as the actual matricies

# upon moving to a new theme a second time, randomly decide whether to do a new traversal,
# whether to copy a previous traversal type list, or whether to straight up use old matrices
# (sometimes, make decision on an instrument by instrument basis)

# to actually implement having the "central arrangement" handler play the indivudal instrument
# melodies in realtime, implement a scheduler that calculates the next melody to play for each
# instrument ahead of time, and provides it upon the "update" message for the instrument.
# Have a threadsafe counter that checks whether all of the instruments are playing the "on deck"
# melody. when the last instrument update message comes in, calculate the next set of melodies

In [75]:
# rather than using hardcoded parameters for rhythmic/harmonic tree traversals,
# use a random walk in the trees, and save the paths taken at each theme

# ignore targetState stuff for now
state = [[random.choice(range(4)) for i in range(6)] for j in range(4)]

def stateDiff(currentState, targetState):
    numParams = len(currentState[0])
    numInst = len(currentState)
    stateDiff = [[0]*numParams for i in range(numInst)] #True means different
    samePairs = set()
    diffPairs = set()
    if targetState is not None:
        for i in range(len(currentState)):
            for j in range(len(currentState[i])):
                stateDiff[i][j] == currentState[i][j] != targetState[i][j]
                if stateDiff[i][j]:
                    diffPairs.addd((i,j))
                else:
                    samePairs.add((i,j))
    return (stateDiff, samePairs, diffPairs)
    
def transformSongMatrix(currentState, targetState=None, towardsTarget=False, pCoordinatedChange=0.2):
    newState = copy.deepcopy(currentState)
    numParams = len(currentState[0])
    numInst = len(currentState)
    instruments = [random.choice([True, False]) for i in range(numInst)]
    stateDiff = [[0]*numParams for i in range(numInst)] #True means different
    samePairs = set()
    diffPairs = set()
    if targetState is not None:
        stateDiff, sampePairs, diffPairs = stateDiff(currentState, targetState)
    coordinatedChange = random.uniform(0, 1) < pCoordinatedChange
    coordinatedParamsToChange = [random.choice([True, False]) for i in range(numParams)]
    transformationType = []
    for i in range(len(currentState)):
        paramsToChange = [random.choice([True, False]) for k in range(numParams)]
        p2c = coordinatedParamsToChange if coordinatedChange else paramsToChange
        transformationType.append([p2c if instruments[i] else False])
        for j in range(len(currentState[i])):
            if instruments[i] and p2c[j]:
                currentVal = currentState[i][j]
                currentState[i][j] = random.choice([set(range(numInst)) - set([currentVal])])
    return (newState, transformationType)

transformSongMatrix(state)

([[3, 2, 3, 1, 2, 1],
  [3, 1, 1, 2, 1, 3],
  [3, 0, 0, 0, 2, 2],
  [3, 0, 0, 1, 0, 0]],
 [[[False, False, False, True, True, False]],
  [False],
  [[True, False, False, False, True, True]],
  [[False, False, True, True, True, True]]])

In [ ]:
class SongState:
    def __init__(self, sectionPoints):
        self.sectionPoints = sectionPoints
        self.sectionVariations = [[]] #for each section save
        self.stepsInSection = random.randint(4,8)
        #roots
        #rhythms
        #melodyTree root
        #rhythmTree root
    
    def doStep(self):
        if self.stepsInSection == 0:
            self.stepsInSection = random.randint(4,8)
            self.switchSection()
            self.sectionVariants.append([])
            return getMelodies()
        else:
            newState = transformSongState()
            self.sectionVariants[-1].append(newState)
    
    def switchSection(self):
        return
    
    def generateMelody(self, instIndex):
        return 
    
    def getMelodies(self, )
    
def transformSongState(arrangementTree, melodyTree, rhythmTree):
    return () #some tuple

In [ ]:
class 

In [ ]:
random.uniform(0, 1)

In [ ]:
loop = [[0.0, 60, 105, 0, 'on'],
 [0.20402940171402, 60, 0, 0, 'off'],
 [0.54597059828598, 60, 106, 0, 'on'],
 [0.23738891228436, 60, 0, 0, 'off'],
 [0.51261108771564, 60, 104, 0, 'on'],
 [0.21680734841448, 60, 0, 0, 'off'],
 [0.53319265158552, 60, 103, 0, 'on'],
 [0.21780347927461, 60, 0, 0, 'off'],
 [0.53219652072539, 60, 108, 0, 'on'],
 [0.16372601843477, 60, 0, 0, 'off'],
 [0.33627398156523, 60, 91, 0, 'on'],
 [0.16410705546696, 60, 0, 0, 'off'],
 [0.33589294453304, 60, 106, 0, 'on'],
 [0.18537746576249, 60, 0, 0, 'off'],
 [0.56462253423751, 60, 103, 0, 'on'],
 [0.17460586809761, 60, 0, 0, 'off'],
 [0.57539413190239, 60, 105, 0, 'on'],
 [0.14310510345877, 60, 0, 0, 'off'],
 [0.35689489654123, 60, 104, 0, 'on'],
 [0.12087395538379, 60, 0, 0, 'off'],
 [0.12912604461621, 60, 91, 0, 'on'],
 [0.17366323724332, 60, 0, 0, 'off'],
 [0.32633676275668, 60, 105, 0, 'on'],
 [0.16298597599166, 60, 0, 0, 'off'],
 [0.08701402400834, 60, 95, 0, 'on'],
 [0.17452761168374, 60, 0, 0, 'off'],
 [0.32547238831626, 60, 116, 0, 'on'],
 [0.19636576324069, 60, 0, 0, 'off'],
 [0.30363423675931, 0, 0, 0, 'timeAfterLastHit']]
loop2 = [[0.0, 63, 98, 0, 'on'],
 [0.08077396240185, 63, 0, 0, 'off'],
 [0.66922603759815, 62, 107, 0, 'on'],
 [0.14656983480671, 62, 0, 0, 'off'],
 [0.60343016519329, 60, 103, 0, 'on'],
 [0.14721199971888, 60, 0, 0, 'off'],
 [0.60278800028112, 62, 110, 0, 'on'],
 [0.15809415775112, 62, 0, 0, 'off'],
 [0.59190584224888, 63, 110, 0, 'on'],
 [0.14687753298239, 63, 0, 0, 'off'],
 [0.60312246701761, 65, 101, 0, 'on'],
 [0.17093161197243, 65, 0, 0, 'off'],
 [0.57906838802757, 67, 107, 0, 'on'],
 [0.14678211657241, 67, 0, 0, 'off'],
 [0.60321788342759, 65, 98, 0, 'on'],
 [0.13548617451193, 65, 0, 0, 'off'],
 [0.61451382548807, 63, 109, 0, 'on'],
 [0.16879031485729, 63, 0, 0, 'off'],
 [0.58120968514271, 62, 110, 0, 'on'],
 [0.15891339785867, 62, 0, 0, 'off'],
 [0.59108660214133, 60, 108, 0, 'on'],
 [0.14645264142581, 60, 0, 0, 'off'],
 [0.35354735857419, 0, 0, 0, 'timeAfterLastHit']]

In [ ]:
for c in itertools.chain(*[[[1], [1], [1]], [[2], [2], [2]]]):
    print c